In [121]:
!pip install torch torchvision torchsummary

You should consider upgrading via the '/home/takumi/.pyenv/versions/3.8.8/bin/python3.8 -m pip install --upgrade pip' command.


In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import os
import shutil
import glob
import os

from PIL import Image
import torch.utils as utils
from torchvision import models
from torchsummary import summary

In [3]:
# 畳み込み層の出力サイズを計算する
def conv_shape(input_size, padding, stride, kernel_size):
	return (input_size + 2*padding - kernel_size)//stride + 1

size = conv_shape(input_size=28, padding=0, stride=1, kernel_size=3)
size = conv_shape(input_size=size, padding=0, stride=1, kernel_size=3)
print(size)

24


In [66]:
12*12*64

9216

In [6]:
USE_MNIST = True
CHANNEL = 1
INPUT_SIZE = 28
NUM_CLASSES = 10
if USE_MNIST:
    ## Mnist
    INPUT_PATH = "/tmp/inputs/data"
    OUTPUT_PATH = "/tmp/result/data"


    # Mnistのデータセット
    batch_size = 32
    test_batch_size = 32
    train_kwargs = {"batch_size": batch_size}
    test_kwargs = {"batch_size": test_batch_size}

    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    )
    # データセットを準備する
    dataset1 = datasets.MNIST(
        INPUT_PATH, train=True, download=True, transform=transform
    )
    dataset2 = datasets.MNIST(INPUT_PATH, train=False, transform=transform)
    train_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    if True:
        for data, labels in train_loader:
            print(data.shape, labels.shape)
            break

torch.Size([32, 1, 28, 28]) torch.Size([32])


In [9]:
## 自作データ
USE_TAKUMI_CLASSIFY = False
CHANNEL = 3
INPUT_SIZE = 256
NUM_CLASSES = 2

if USE_TAKUMI_CLASSIFY:
    class MyDatasets(torch.utils.data.Dataset):
        def __init__(self, root_dir, key, transform):
            self.transform = transform
            self.data = []
            self.labels = []
            name_to_label = {"takumi": 0, "other": 1}

            target_path_list = []
            target_dir = os.path.join(root_dir, key, "**/*")
            print(target_dir)

            for path in glob.glob(target_dir):
                name = os.path.basename(os.path.dirname(path))
                label = name_to_label[name]

                self.data.append(path)
                self.labels.append(label)

        def __len__(self):
            return len(self.data)

        def __getitem__(self, index):
            img_path = self.data[index]
            label = self.labels[index]

            img = Image.open(img_path).convert("RGB")

            img = self.transform(img)

            return img, label

    # DATASET_PATH = "/home/jovyan/data"
    DATASET_PATH = "/mnt/data/lake/TakumiClassify"

    transform = transforms.Compose([transforms.Resize((INPUT_SIZE, INPUT_SIZE)), transforms.ToTensor()])
    train_dataset = MyDatasets(DATASET_PATH, "train", transform)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8)
    test_dataset = MyDatasets(DATASET_PATH, "test", transform)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8)


    if True:
        for data, labels in train_loader:
            print(data.shape, labels.shape)
            break

In [7]:
## 自作モデル
USE_MY_MODEL=False
if USE_MY_MODEL:
    class Net(nn.Module):
        def __init__(self, channel=1, input_size=28, num_class=10):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(channel, 32, 3, 1) # 入力チャンネルは、画像タイプに合わせて変える
            self.conv2 = nn.Conv2d(32, 64, 3, 1)
            self.dropout1 = nn.Dropout(0.25)
            self.dropout2 = nn.Dropout(0.5)
            fc1_in_features = self._calc_fc1_in_features(input_size)
            self.fc1 = nn.Linear(fc1_in_features, 128)
            self.fc2 = nn.Linear(128, num_class)

        def forward(self, x):
            x = self.conv1(x)# 28
            x = F.relu(x)
            x = self.conv2(x)# 24
            x = F.relu(x)
            x = F.max_pool2d(x, 2)# 12
            x = self.dropout1(x)
            x = torch.flatten(x, 1)
            x = self.fc1(x)
            x = F.relu(x)
            x = self.dropout2(x)
            x = self.fc2(x)
            output = F.log_softmax(x, dim=1)
            return output

        def _conv_shape(self, input_size, padding, stride, kernel_size):
            return (input_size + 2*padding - kernel_size)//stride + 1

        def _calc_fc1_in_features(self, input_size):
            # NOTE: モデル構造に依存するので注意
            conv1_out_size = self._conv_shape(input_size=input_size, padding=0, stride=1, kernel_size=3)
            conv2_out_size = self._conv_shape(input_size=conv1_out_size, padding=0, stride=1, kernel_size=3)
            max_pool2d_out_size = conv2_out_size/2
            return int(max_pool2d_out_size * max_pool2d_out_size * 64)


In [5]:
## デバッグ
if USE_MY_MODEL:
	input_size = 64
	model = Net(channel=3, input_size=input_size)

	input_img = torch.rand(1, 3, input_size, input_size)
	res = model(input_img)
	print(input_img.shape)
	print(res.shape)
	print(res)


torch.Size([1, 3, 64, 64])
torch.Size([1, 2])
tensor([[-0.6734, -0.7133]], grad_fn=<LogSoftmaxBackward0>)


In [12]:
## VGG
USE_VGG=True
if USE_VGG:
	def get_vgg():
		vgg19 = models.vgg19(pretrained=True)
		vgg19.classifier[6] = nn.Linear(in_features=4096, out_features=NUM_CLASSES)
		return vgg19




In [11]:
## デバッグ
if USE_VGG:
	input_size = 224

	input_img = torch.rand(1, 3, input_size, input_size)
	model = get_vgg()
	res = model(input_img)
	print(res.shape)

	model.to("cuda")
	summary(model, (3,input_size,input_size))

NameError: name 'USE_VGG' is not defined

In [13]:
# 学習
def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item(),
                )
            )

# 評価
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(
                output, target, reduction="sum"
            ).item()  # sum up batch loss
            pred = output.argmax(
                dim=1, keepdim=True
            )  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print(
        "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss,
            correct,
            len(test_loader.dataset),
            100.0 * correct / len(test_loader.dataset),
        )
    )

In [16]:
USE_MY_MODEL

True

In [14]:
## 学習
epochs = 5
log_interval = 10000

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# if device == "cuda":
# 	cuda_kwargs = {"num_workers": 1, "pin_memory": True, "shuffle": True}

if USE_MY_MODEL:
	model = Net(channel=CHANNEL, input_size=INPUT_SIZE, num_class=NUM_CLASSES)
	model.to(device)
elif USE_VGG:
	model = get_vgg()
	model.to(device)


optimizer = optim.Adadelta(model.parameters(), lr=0.001)

scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(1, epochs + 1):
	train(log_interval, model, device, train_loader, optimizer, epoch)
	test(model, device, test_loader)
	scheduler.step()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.315488

Test set: Average loss: 1.0988, Accuracy: 8181/10000 (82%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.430103

Test set: Average loss: 0.6332, Accuracy: 8621/10000 (86%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.791315

Test set: Average loss: 0.5133, Accuracy: 8755/10000 (88%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.577979

Test set: Average loss: 0.4646, Accuracy: 8830/10000 (88%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.789992

Test set: Average loss: 0.4399, Accuracy: 8865/10000 (89%)



In [44]:
device

device(type='cuda')

In [45]:
128*2


256